In [1]:
! pip install transformers

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [3]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
! pip3 install keras --upgrade

In [6]:
df = pd.read_csv('./Ethos_Dataset_Multi_Label.csv',sep=';')

In [5]:
df

,comment,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
0,You should know women's sports are a joke,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,You look like Sloth with deeper Down’s syndrome,0.000000,0.833333,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,You look like Russian and speak like Indian. B...,1.000000,0.500000,0.142857,0.142857,0.714286,0.000000,0.000000,0.000000
3,"Women deserve to be abused, I guess.",0.666667,0.166667,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Women are made for making babies and cooking d...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
428,All people are fucked up because of these quee...,0.000000,0.000000,0.571429,0.000000,0.000000,0.000000,0.000000,0.571429
429,Actually THIS WEIRD PEOPLE are HARMFUL! They n...,0.000000,1.000000,0.000000,0.000000,0.333333,0.666667,0.000000,0.000000
430,ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.857143,0.000000
431,a homo? so you fuck your brother instead of yo...,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [6]:
df.duplicated

<bound method DataFrame.duplicated of                                                comment  violence  \
0            You should know women's sports are a joke  0.000000   
1      You look like Sloth with deeper Down’s syndrome  0.000000   
2    You look like Russian and speak like Indian. B...  1.000000   
3                 Women deserve to be abused, I guess.  0.666667   
4    Women are made for making babies and cooking d...  0.000000   
..                                                 ...       ...   
428  All people are fucked up because of these quee...  0.000000   
429  Actually THIS WEIRD PEOPLE are HARMFUL! They n...  0.000000   
430  ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...  0.000000   
431  a homo? so you fuck your brother instead of yo...  0.000000   
432  (((They))) will be chased out of every white c...  0.333333   

     directed_vs_generalized    gender      race  national_origin  disability  \
0                   0.000000  1.000000  0.000000         0.00000

In [7]:
# List of columns to process

columns_to_process = ['violence', 'gender',	'race',	'national_origin',	'disability',	'religion',	'sexual_orientation']

# Process each cell in the specified columns
for column in columns_to_process:
    df[column] = df[column].apply(lambda x: 0 if x < 0.5 else 1 if x > 0.5 else 1)

In [7]:
df.drop('directed_vs_generalized', inplace=True, axis=1)

In [10]:
# List to store column indices where 1s are located in each row
columns_with_ones = []

# Dictionary to map column names to corresponding indices (1 to 7)
column_index_mapping = {
    'violence': 1,
    'gender': 2,
    'race': 3,
    'national_origin': 4,
    'disability': 5,
    'religion': 6,
    'sexual_orientation': 7
}

# Iterate over rows and columns to find 1s and store column indices
for _, row in df.iterrows():
    indices = [column_index_mapping[column] for column in row.index if row[column] == 1]
    columns_with_ones.append(','.join(map(str, indices)))

# Create a new column in the DataFrame and add column indices with 1s
df['Labels'] = columns_with_ones


In [9]:
df

,comment,violence,directed_vs_generalized,gender,race,national_origin,disability,religion,sexual_orientation
0,You should know women's sports are a joke,0,0.000000,1,0,0,0,0,0
1,You look like Sloth with deeper Down’s syndrome,0,0.833333,0,0,0,1,0,0
2,You look like Russian and speak like Indian. B...,1,0.500000,0,0,1,0,0,0
3,"Women deserve to be abused, I guess.",1,0.166667,1,0,0,0,0,0
4,Women are made for making babies and cooking d...,0,0.000000,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
428,All people are fucked up because of these quee...,0,0.000000,1,0,0,0,0,1
429,Actually THIS WEIRD PEOPLE are HARMFUL! They n...,0,1.000000,0,0,0,1,0,0
430,ABU DA BI ABA DU BA BA HAMUD! See? I’m an arab...,0,0.000000,0,0,0,0,1,0
431,a homo? so you fuck your brother instead of yo...,0,0.666667,0,0,0,0,0,1


In [8]:
df.isna().sum()

comment               0
violence              0
gender                0
race                  0
national_origin       0
disability            0
religion              0
sexual_orientation    0
dtype: int64

In [8]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [10]:
df.drop('directed_vs_generalized', inplace=True, axis=1)

In [11]:
# Tokenize input text and convert to BERT embeddings
input_ids = []
for text in df['comment']:
    input_ids.append(tokenizer.encode(text, add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
# Process input in batches
batch_size = 8
bert_outputs = []
for i in range(0, len(input_ids), batch_size):
    batch_input_ids = np.array(input_ids[i:i + batch_size])
    batch_bert_output = bert_model(batch_input_ids)[0]
    bert_outputs.append(batch_bert_output)


In [13]:
# Concatenate batch outputs
bert_output = np.concatenate(bert_outputs, axis=0)

In [14]:
print(input_ids)

[[101, 2017, 2323, 2113, 2308, 1005, 1055, 2998, 2024, 1037, 8257, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
# Convert labels to multi-label format
label_columns = ['violence', 'gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation']
labels = df[label_columns].values



In [25]:
print(labels)

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]]


In [21]:
# Build neural network model for multi-label classification
input_layer = Input(shape=(bert_output.shape[1], bert_output.shape[2]), dtype=tf.float32)
dense_layer = Dense(128, activation='relu')(input_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(7, activation='sigmoid')(dropout_layer)  # Sigmoid for multi-label classification
model = Model(inputs=input_layer, outputs=output_layer)

In [22]:
# Define custom loss function to handle varying label shapes
def custom_loss(y_true, y_pred):
    y_true = tf.squeeze(y_true, axis=1)  # Remove the singleton dimension from labels if present
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

In [23]:
# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [24]:
# Train the model
model.fit(bert_output, labels, epochs=5, batch_size=1)

Epoch 1/5


ValueError: ignored

In [ ]:
# Tokenize input text and convert to BERT embeddings
input_ids = []
for text in df['text']:
    input_ids.append(tokenizer.encode(text, add_special_tokens=True, max_length=512, pad_to_max_length=True, truncation=True))

input_ids = np.array(input_ids)
bert_output = bert_model(input_ids)[0]

# Convert labels to multi-label format
label_columns = ['violence', 'gender', 'race', 'national_origin', 'disability', 'religion', 'sexual_orientation']
labels = df[label_columns].values

# Build neural network model for multi-label classification
input_layer = Input(shape=(bert_output.shape[1], bert_output.shape[2]), dtype=tf.float32)
dense_layer = Dense(128, activation='relu')(input_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(len(label_columns), activation='sigmoid')(dropout_layer)  # Sigmoid for multi-label classification
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
model.fit(bert_output, labels, epochs=5, batch_size=1)

Import Libraries: Import necessary libraries including TensorFlow and Hugging Face's Transformers library.

Initialize BERT Tokenizer and Model: Initialize the BERT tokenizer and model from Hugging Face's Transformers library.

Tokenize Input Text: Tokenize the input text using the BERT tokenizer. The encode function converts the text to token IDs. The pad_to_max_length and truncation parameters ensure that the input has a fixed length suitable for BERT.

Pass Input through BERT Model: Pass the tokenized input through the BERT model to obtain BERT embeddings for the input text.

Build Neural Network Model: Build a neural network model. In this example, a dense layer with softmax activation is added on top of the BERT output. The output shape is set to 2, representing the two classes: hate speech and non-hate speech.

Compile the Model: Compile the model with sparse categorical cross-entropy loss and the Adam optimizer.

Example Labels: Define example labels (0 for non-hate speech, 1 for hate speech). In a real application, you would have a dataset with corresponding labels.

Train the Model: Train the model. In this example, the same BERT output is used as both input and labels for simplicity. In a real scenario, you would have separate input and label data.

Step 1: Importing Necessary Libraries
First, we need to import the necessary libraries. We’ll be using the Huggingface Transformers library, which provides a straightforward interface for working with BERT and other transformer models.

⚠ This code is experimental content and was generated by AI. Please refer to this code as experimental only since we cannot currently guarantee its validity

from transformers import BertTokenizer
Step 2: Loading the Pretrained Tokenizer
Next, we load the pretrained BERT tokenizer. For this example, we’ll use the ‘bert-base-uncased’ model, but you can replace this with any BERT model you prefer.

⚠ This code is experimental content and was generated by AI. Please refer to this code as experimental only since we cannot currently guarantee its validity

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
Step 3: Adding New Tokens
Now, we’re ready to add new tokens. The add_tokens method makes this easy. It takes a list of new tokens and returns the number of tokens actually added to the vocabulary.

⚠ This code is experimental content and was generated by AI. Please refer to this code as experimental only since we cannot currently guarantee its validity

new_tokens = ['newtoken1', 'newtoken2']
num_added_tokens = tokenizer.add_tokens(new_tokens)
print('Number of tokens added:', num_added_tokens)
Step 4: Resizing Token Embeddings
After adding new tokens, we need to resize the token embeddings of the BERT model to match the new size of the vocabulary. This is done using the resize_token_embeddings method.

⚠ This code is experimental content and was generated by AI. Please refer to this code as experimental only since we cannot currently guarantee its validity

from transformers import BertModel

model = BertModel.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))
Step 5: Saving the Tokenizer
Finally, we save the updated tokenizer for future use. The save_pretrained method saves both the tokenizer configuration and the vocabulary, which includes the new tokens.

⚠ This code is experimental content and was generated by AI. Please refer to this code as experimental only since we cannot currently guarantee its validity

tokenizer.save_pretrained('path_to_directory')
Conclusion
Adding new tokens to the Huggingface BERT tokenizer is a straightforward process that can greatly improve the efficiency of your NLP tasks. By following these steps, you can customize the tokenizer to better suit your specific needs.

Remember, when adding new tokens, it’s important to also update your model’s token embeddings to match the new vocabulary size. And don’t forget to save your updated tokenizer for future use!

BERT and its tokenizer are powerful tools in the NLP toolkit. With the ability to add new tokens, you can make these tools even more powerful and tailored to your specific use cases.

Keywords: Huggingface, BERT, Tokenizer, Add New Tokens, NLP, Transformers, Python, Data Science, Machine Learning

Meta Description: Learn how to add new tokens to the Huggingface BERT tokenizer in this comprehensive guide. Ideal for data scientists and machine learning practitioners working with NLP tasks.